In [1]:
import boto3
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

# boto3를 통해 자동으로 자격증명 가져오기
# ~/.aws/credentials를 미리 만들어놔야 함
session = boto3.Session()
credentials = session.get_credentials()
aws_access_key_id = credentials.access_key
aws_secret_access_key = credentials.secret_key

In [2]:
# Create a Spark session with your AWS Credentials
conf = (
    SparkConf()
    .setAppName("sparkQLwithS3") # replace with your desired name
    .set("spark.jars.packages", \
         "io.delta:delta-core_2.12:2.3.0"\
         ",org.apache.hadoop:hadoop-aws:3.3.2"\
         ",org.mariadb.jdbc:mariadb-java-client:3.3.3"\
        )
    .set("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .set("spark.sql.shuffle.partitions", "4") # default is 200 partitions which is too many for local
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("com.amazonaws.services.s3.enableV4", "true")
    .set("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com")
    # .set("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.profile.ProfileCredentialsProvider")
    .set("spark.hadoop.fs.s3a.access.key", aws_access_key_id)
    .set("spark.hadoop.fs.s3a.secret.key", aws_secret_access_key)
    .setMaster("spark://34.125.136.103:30077")
)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/root/miniconda3/envs/spark341/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.mariadb.jdbc#mariadb-java-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5ec788b3-6315-48b7-bdb3-4c3ada0e091a;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.mariadb.jdbc#mariadb-java-client;3.3.3 in central
	found com.github.waffle#waffle-jna;3.3.0 in central
	found net.java.dev.jna#jna;5.13.0 in central
	found net.java.dev.jna#jna-platform;5.13.0 in central
	found org.slf4j#jcl-over-slf4j;2.0.7 in central
	found org.slf4j#slf4j-api;2.0.7

In [3]:
df = spark.read.format('json').load('s3a://jolajoayo-spark-0001/spark2-sql/airports/airport-codes.csv.json')

24/04/29 21:39:40 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
df.createOrReplaceTempView("airportcodes")
ssql = "\
    SELECT iso_region, name, type \
    FROM airportcodes \
    WHERE iso_country = 'US' \
"
viewdf = spark.sql(ssql)

print(viewdf.count())
viewdf.show()

ANTLR Tool version 4.8 used for code generation does not match the current runtime version 4.9.3
ANTLR Runtime version 4.8 used for parser compilation does not match the current runtime version 4.9.3
ANTLR Tool version 4.8 used for code generation does not match the current runtime version 4.9.3
ANTLR Runtime version 4.8 used for parser compilation does not match the current runtime version 4.9.3


+----------+--------------------+-------------+
|iso_region|                name|         type|
+----------+--------------------+-------------+
|     US-PA|   Total Rf Heliport|     heliport|
|     US-AK|        Lowell Field|small_airport|
|     US-AL|        Epps Airpark|small_airport|
|     US-AR|Newport Hospital ...|     heliport|
|     US-AZ|      Cordes Airport|small_airport|
|     US-CA|Goldstone /Gts/ A...|small_airport|
|     US-CO|          Cass Field|small_airport|
|     US-FL| Grass Patch Airport|small_airport|
|     US-FL|  Ringhaver Heliport|     heliport|
|     US-FL|   River Oak Airport|small_airport|
|     US-GA|    Lt World Airport|small_airport|
|     US-GA|    Caffrey Heliport|     heliport|
|     US-HI|  Kaupulehu Heliport|     heliport|
|     US-ID|Delta Shores Airport|small_airport|
|     US-IN|Bailey Generation...|     heliport|
|     US-IL|      Hammer Airport|small_airport|
|     US-IN|St Mary Medical C...|     heliport|
|     US-IL|Hayenga's Cant Fi...|small_a

In [6]:
# 접속 정보
jdbcHostname = "34.125.136.103"
jdbcDatabase = "sparkdb"
jdbcPort = 30007
jdbcUrl = "jdbc:mysql://{0}:{1}/{2}?permitMysqlScheme".format(jdbcHostname, jdbcPort, jdbcDatabase)
jdbcTable = "tb_airport_us"
properties = {
  "driver" : "org.mariadb.jdbc.Driver",
  "user"   : "root",
  "password" : "tjehdgml"
}

In [24]:
# tb_airport_us이 없을 경우 새로 만든다.
# mode: overwrite, append, ignore, error, errorifexists
# overwrite: 테이블 싹 지우고 새로 넣기
# error/errorifexist: 테이블만 있어도 에러 발
viewdf.write.format("jdbc") \
                .mode("append") \
                .option("url", jdbcUrl) \
                .option("driver", properties["driver"]) \
                .option("dbtable", jdbcTable) \
                .option("user", properties["user"]) \
                .option("password", properties["password"]) \
                .save()

In [27]:
df = spark.read.format("jdbc") \
                    .option("url", jdbcUrl) \
                    .option("driver", properties["driver"]) \
                    .option("dbtable", jdbcTable) \
                    .option("user", properties["user"]) \
                    .option("password", properties["password"]) \
                    .load()
                    
df.count()

0

In [26]:
# truncate
# 결과는 위 셀로 확인
viewdf.limit(0).write.format("jdbc") \
                .mode("overwrite") \
                .option("url", jdbcUrl) \
                .option("driver", properties["driver"]) \
                .option("dbtable", jdbcTable) \
                .option("user", properties["user"]) \
                .option("password", properties["password"]) \
                .save()